In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from datetime import timedelta
from dotenv import load_dotenv
from scipy.fft import fft, fftfreq
from uuid import UUID

In [ ]:
load_dotenv()

In [ ]:
plt.rcParams["figure.figsize"] = (16, 2)

In [ ]:
connection_string = os.getenv('DATABASE_URL')
day = '2021-02-10'

## Load location data for one day

In [ ]:
location = pd.read_sql_query(f"""
    SELECT uuid, time, atype, speed, sensor_data_count FROM trips_location
    WHERE time::date = '{day}'
    """, connection_string)
# Convert m/s to km/h
location.speed *= 3.6

## Load sensor data for one day

In [ ]:
sensor = pd.read_sql_query(f"""
    SELECT type, uuid, time, sqrt(x*x + y*y + z*z) - 1 AS m FROM trips_sensorsample
    WHERE time::date = '{day}'
    AND type = 'acce'
    """, connection_string)
# acce = sensor[sensor.type == 'acce']
acce = sensor

## Load sensor data combined with closest location data (one day)

In [ ]:
closest = pd.read_sql_query(f"""
    SELECT DISTINCT ON (s.time) s.time AS stime, l.time AS ltime, s.uuid, s.type, sqrt(s.x*s.x + s.y*s.y + s.z*s.z) - 1 AS m, l.atype, l.speed
    FROM trips_sensorsample AS s
    CROSS JOIN trips_location AS l
    WHERE s.uuid = l.uuid
    AND s.type = 'acce'
    -- AND s.uuid = '109ed1e2-f4d4-4874-b741-c62c3e11896e'
    AND s.time::date = '{day}'
    AND l.time::date = '{day}'
    AND greatest(-(s.time - l.time), s.time - l.time) < interval '10 seconds'
    ORDER BY s.time, s.time <-> l.time
    """, connection_string)
closest.speed *= 3.6

## Check reliability of "walking" atype (with cutoff)

In [ ]:
walking = location[(location.atype == 'walking') | (location.atype == 'on_foot')]

In [ ]:
filtered = walking.speed[~np.isnan(walking.speed)]
print(filtered.describe())

In [ ]:
plt.boxplot(filtered, vert=False)
plt.show()
plt.boxplot(filtered, vert=False, showfliers=False)
plt.show()
plt.hist(filtered, bins=20, range=[0, 10])
plt.show()

## Explore day for a UUID

In [ ]:
uuid = '8694706b-7fd4-4b8e-8346-dee2caea7913'
# uuid = '2af9bf5f-2ce5-4c73-854a-b996c15b2d9b'
# uuid = '109ed1e2-f4d4-4874-b741-c62c3e11896e'

In [ ]:
user_acce = acce[acce.uuid == UUID(uuid)]
user_loc = location[location.uuid == UUID(uuid)]

In [ ]:
plt.scatter(user_acce.time, np.full_like(user_acce.time, 'acce'))
plt.scatter(user_loc.time, np.full_like(user_loc.time, 'location'))
plt.show()

## Cluster sensor data into bursts

In [ ]:
def cluster_bursts(df):
    bursts = df.copy()
    burst_column = []
    last_time = None
    current_burst = 0
    for time in bursts.stime:
        if last_time is None or time > last_time + timedelta(minutes=1):
            current_burst += 1
        last_time = time
        burst_column.append(current_burst)
    bursts['burst'] = burst_column
    return bursts

In [ ]:
bursts = cluster_bursts(closest[(closest.type == 'acce') & (closest.uuid == UUID(uuid))])
display(bursts)

## FFT

In [ ]:
def interpolate_burst(burst, interval=0.01):
    new_stime = (burst.stime - burst.stime.min()).dt.total_seconds()
    new_x = np.arange(0, new_stime.max(), interval)
    new_y = np.interp(new_x, new_stime, burst.m)
    return new_x, new_y

def plot_burst_raw(burst):
    plt.plot(burst.stime, burst.m, 'o')
    plt.show()
    
def plot_burst_interpolated(burst):
    new_x, new_y = interpolate_burst(burst)
    plt.plot(new_x, new_y)
    plt.show()

def plot_burst_fft(burst):
    new_x, new_y = interpolate_burst(burst)
    yf = fft(new_y)
    N = len(new_y)
    T = 0.01
    xf = fftfreq(N, T)[:N//2]
    plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
    plt.show()
    
def plot_burst_all(burst):
    plot_burst_raw(burst)
    plot_burst_interpolated(burst)
    plot_burst_fft(burst)

In [ ]:
def surrounding_loc(uuid, earliest, latest):
    df = pd.read_sql_query(f"""
        SELECT uuid, time, atype, speed
        FROM trips_location
        WHERE uuid = '{uuid}'
        AND time >= '{earliest}'
        AND time <= '{latest}'
        """, connection_string)
    df.speed *= 3.6
    return df

# surrounding_loc(uuid, '2021-02-10', '2021-02-11')

In [ ]:
def process_burst(burst):
    print(f"Mean speed among closest location data points: {burst.speed.mean()}")
    print(f"Most common atype among closest location data points: {burst.atype.value_counts().index[0]}")
    plot_burst_all(burst)
    print("Surrounding location data:")
    uuids = burst.uuid.unique()
    assert len(uuids) == 1
    uuid = uuids[0]
    display(surrounding_loc(uuid,
                            burst.stime.min() - timedelta(seconds=10),
                            burst.stime.max() + timedelta(seconds=10)))

In [ ]:
for i in bursts.burst.unique():
    print(f"Burst {i} on {day} for {uuid}")
    process_burst(bursts[bursts.burst == i])
    print()

### Get closest location data point for burst

In [ ]:
burst_middle_time = burst.time.median()
user_loc.time.get_loc(burst_middle_time, method='nearest')
# nearest_loc = user_loc.iloc[user_loc.index.get_loc(burst_middle_time, method='nearest')]

## Sample statistics

In [ ]:
sample.m.describe()

In [ ]:
plt.hist(sample.m, bins=20, range=[-1,1])
plt.show()